# Set up

In [1]:
import numpy as np
import pandas as pd

import re

# import csv
import pickle

# from collections import defaultdict

import requests

# from datetime import datetime
# import locale # to allow date parsing for dates in Dutch

# from collections import Counter

# import matplotlib.pyplot as plt

# import copy

from workalendar.europe import Belgium
from datetime import timedelta

In [2]:
# show all outputs of cell, not merely of last line (i.e. default of Jupyter Notebook)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Read in list of members and their parties to later on do mapping
with open('../data/parlementsleden.pkl', 'rb') as file:
    parlementsleden_all_dict = pickle.load(file)

Interesting fields for questions:
* /schv/lijst: Schriftelijke vragen op basis van id's van de vragen
* /vi/lijst: Lijst van vragen en interpellaties op basis van id's  van de initiatieven

# Obtain relevant id's

First we need to obtain the id's of all relevant questions. There is no straightforward way to obtain this. Contact with the administration of the Flemish Parliament learns that the best way to do this is to launch a search query for the questions, and then use the 'opendata' tags to obtain the relevant id's of all results. 

Such a query can f.e. be the following: 'https://ws.vlpar.be/api/search/query/+inmeta:zittingsjaar=2023-2024&requiredfields=paginatype:Parlementair%20document.aggregaat:Vraag%20of%20interpellatie.initiatief:Schriftelijke%20vraag?collection=vp_collection&sort=date&max=100&page=1'. This lists all written questions ('Schriftelijke vragen') within the working year 2023-2024. However, it seems not possible to obtain all results at one go. The amount of search results that can be displayed on the first page is limited (f.e. not possible to list more than 100 results on the same page). 

Nevertheless, a count is provided. so it is possible to iterate through all pages with each 100 results to obtain all search results. 

First, we define various functions to obtain these search results and id's. 

It seems not possible to obtain all data for an entire parliamentary term ('zittingsperiode'). however, it is possible to iterate through various parliamentary years ('zittingsjaren'.

In [4]:
def get_request(url_query:str):
    """
    Parse data from search query using url
    """
    # Make the GET request, specifing you want to use json as header, instead of xml
    response = requests.get(url_query, headers = {"Accept": "application/json"})

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()  # Parse JSON response
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    return data

In [5]:
def get_written_question_details(url_query: str):
    """
    Return id's of all questions by iterating over all pages
    """
    url_page = f"{url_query}1" # Initialize page at 1
    # obtain output of first page of search results
    first_page = get_request(url_page)

    # Obtain amount of total results for query
    total = int(first_page['count'])
    print("Total amount of questions: ", total)

    # obtain amount of pages are needed to display all results 
    # (i.e. amount of full pages of 100 results + page with remainder)
    pages = total//100 + 1

    
    questions_details_list = []
    
    # Iterate over all pages, starting from page 1
    for page in range(1,pages+1):
        url_page = f"{url_query}{page}" # initialise URL based on relevant page
        data = get_request(url_page)
        
        # iterate over each question in the results, extract relevant details and store in dict
        for question in data["result"]:
            question_details_dict = {
                'id': re.search(r'\d+$', question['url']).group(),
                # analysis of the raw data and the main website indidate that 'themadatum' is the date when a question was asked, and 'statusdatumSchvBeantwoord' the date of answering. 
                'datum gesteld': next((tag['value'] for tag in question['metatags']['metatag'] if tag['name'] == 'themadatum'), None), 
                'datum beantwoord': next((tag['value'] for tag in question['metatags']['metatag'] if tag['name'] == 'statusdatumSchvBeantwoord'), None),
                'minister': next((tag['value'] for tag in question['metatags']['metatag'] if tag['name'] == 'minister'), None),
                'onderwerp': next((tag['value'] for tag in question['metatags']['metatag'] if tag['name'] == 'onderwerp'), None),
                'documenttype': next((tag['value'] for tag in question['metatags']['metatag'] if tag['name'] == 'documenttype'), None),
                'thema': next((tag['value'] for tag in question['metatags']['metatag'] if tag['name'] == 'thema'), None),
                'vraagsteller': next((tag['value'] for tag in question['metatags']['metatag'] if tag['name'] == 'vraagsteller'), None),
                # 'url': next((tag['value'] for tag in question if tag['name'] == 'url'), None),
                'url': question["url"],
            }
            # Append the dictionary to the list
            questions_details_list.append(question_details_dict)
    
    # Create a DataFrame from the list of dictionaries
    questions_details_df = pd.DataFrame(questions_details_list) 
    return questions_details_df

Then we apply the functions to actually obtain the details for each parliamentary year.

In [6]:
# List of 'zittingsjaar' values
zittingsjaar_values = ['2019-2020', '2020-2021', '2021-2022', '2022-2023', '2023-2024']

# Base URL template
base_url = "https://ws.vlpar.be/api/search/query/+inmeta:zittingsjaar={}&requiredfields=paginatype:Parlementair%20document.aggregaat:Vraag%20of%20interpellatie.initiatief:Schriftelijke%20vraag?collection=vp_collection&sort=date&max=100&page="

details_questions_term_list = [] # initialize list for dfs

# Iterate over 'zittingsjaar' values
for zittingsjaar in zittingsjaar_values:
    print("Zittingsjaar: ", zittingsjaar)
    # Construct the dynamic URL
    dynamic_url = base_url.format(zittingsjaar)

    # Obtain dataframe with details for each parliamentary year
    details_questions_df = get_written_question_details(dynamic_url)
    # append to list
    details_questions_term_list.append(details_questions_df)

# concatenate to single df
details_questions_term_df = pd.concat(details_questions_term_list, ignore_index=True)

Zittingsjaar:  2019-2020
Total amount of questions:  6403
Zittingsjaar:  2020-2021
Total amount of questions:  7869
Zittingsjaar:  2021-2022
Total amount of questions:  7137
Zittingsjaar:  2022-2023
Total amount of questions:  7345
Zittingsjaar:  2023-2024
Total amount of questions:  1146


In [7]:
# Inspect results
details_questions_term_df.shape
details_questions_term_df.head()
details_questions_term_df.tail()

(29900, 9)

,id,datum gesteld,datum beantwoord,minister,onderwerp,documenttype,thema,vraagsteller,url
0,1363595,2020-01-29,2021-02-05,"Beke, Wouter",Baarmoederhalskanker - Regionale verschillen...,Vraag en antwoord,Gezondheid,"Saeys, Freya",https://www.vlaamsparlement.be/parlementaire-d...
1,1424087,2020-09-24,2020-11-16,"Crevits, Hilde",Bedrijfssteun - Evaluatie (3),Vraag en antwoord,Economie,"Vande Reyde, Maurits",https://www.vlaamsparlement.be/parlementaire-d...
2,1424234,2020-09-28,2020-11-01,"Weyts, Ben",Boekencheck 2020 - Resultaten,Vraag en antwoord,Onderwijs en Vorming,"Tavernier, Annabel",https://www.vlaamsparlement.be/parlementaire-d...
3,1424254,2020-09-28,2020-10-29,"Somers, Bart",Gemeente- en OCMW-raadsleden - Inzagerecht v...,Vraag en antwoord,Lokale Overheden,"Warnez, Brecht",https://www.vlaamsparlement.be/parlementaire-d...
4,1423090,2020-09-22,2020-10-27,"Weyts, Ben",Lerarenopleiding - Instaptoets,Vraag en antwoord,Onderwijs en Vorming,"Vandromme, Loes",https://www.vlaamsparlement.be/parlementaire-d...


,id,datum gesteld,datum beantwoord,minister,onderwerp,documenttype,thema,vraagsteller,url
29895,1765819,2023-10-02,2023-10-06,"Peeters, Lydia",Grensoverschrijdende mobiliteitsprojecten - ...,Vraag en antwoord,Openbare werken,"Keulen, Marino",https://www.vlaamsparlement.be/parlementaire-d...
29896,1765821,2023-10-23,2023-10-06,"Peeters, Lydia",IJzeren Rijn - Stand van zaken,Vraag en antwoord,Mobiliteit en Verkeer,"Keulen, Marino",https://www.vlaamsparlement.be/parlementaire-d...
29897,1767239,2023-10-04,2023-10-05,"Demir, Zuhal",Lage-emissiezones (LEZ's) - Wijzigingsbesluit,Vraag en antwoord,Mobiliteit en Verkeer,"Bex, Stijn",https://www.vlaamsparlement.be/parlementaire-d...
29898,1765825,2023-10-02,2023-10-26,"Peeters, Lydia",Keuring oldtimers - Knelpunten,Vraag en antwoord,Mobiliteit en Verkeer,"Keulen, Marino",https://www.vlaamsparlement.be/parlementaire-d...
29899,1765823,2023-10-02,2023-10-26,"Peeters, Lydia",Evolutie verkeersslachtoffers - Beleid,Vraag en antwoord,Mobiliteit en Verkeer,"Keulen, Marino",https://www.vlaamsparlement.be/parlementaire-d...


Then we modify the dataframe to prepare it for easier handling by the dash application:

* Switch the first and last names of the relevant (i.e. members of parliament and ministers). This not only improves the readability but also ensures they are more easily matched with othere data structures were names are fromatted as [first name, last name].
* Map the relevant party to each member
* Cast the dates to the right format
* Include a column on the time it took to answer a question
* Merge the values of the columns 'onderwerp' and 'url' into a markdown representation that can be used later on in dash application. 

In [8]:
# Function to switch order and remove comma
def switch_order_and_remove_comma(name):
    first_name, last_name = map(str.strip, name.split(','))
    return f'{last_name} {first_name}'

In [9]:
# Apply the function to the 'minister' and 'vraagsteller' column
details_questions_term_df['minister'] = details_questions_term_df['minister'].apply(switch_order_and_remove_comma)
details_questions_term_df['vraagsteller'] = details_questions_term_df['vraagsteller'].apply(switch_order_and_remove_comma)

In [10]:
# Function to map member to party
def map_member_to_party(member):
    for key, value in parlementsleden_all_dict.items():
        if value[0] == member:
            return value[1]
    return None  # Handle the case where member is not found

In [11]:
# Map member to party
details_questions_term_df['vraagsteller_partij'] = details_questions_term_df['vraagsteller'].map(map_member_to_party)

In [12]:
# Finally we cast the 'datum gesteld' and 'datum beantwoord' columns to a datetime format
details_questions_term_df['datum gesteld'] = pd.to_datetime(details_questions_term_df['datum gesteld'], format='%Y-%m-%d')
details_questions_term_df['datum beantwoord'] = pd.to_datetime(details_questions_term_df['datum beantwoord'], format='%Y-%m-%d')

# Obtain time difference (do this before turning to date() - see below - to avoid errors)
details_questions_term_df['termijn antwoord'] = (details_questions_term_df['datum beantwoord'] - 
                                                 details_questions_term_df['datum gesteld']
                                                ).dt.days

In [13]:
details_questions_term_df.head()

,id,datum gesteld,datum beantwoord,minister,onderwerp,documenttype,thema,vraagsteller,url,vraagsteller_partij,termijn antwoord
0,1363595,2020-01-29,2021-02-05,Wouter Beke,Baarmoederhalskanker - Regionale verschillen...,Vraag en antwoord,Gezondheid,Freya Saeys,https://www.vlaamsparlement.be/parlementaire-d...,Open Vld,373.0
1,1424087,2020-09-24,2020-11-16,Hilde Crevits,Bedrijfssteun - Evaluatie (3),Vraag en antwoord,Economie,Maurits Vande Reyde,https://www.vlaamsparlement.be/parlementaire-d...,Open Vld,53.0
2,1424234,2020-09-28,2020-11-01,Ben Weyts,Boekencheck 2020 - Resultaten,Vraag en antwoord,Onderwijs en Vorming,Annabel Tavernier,https://www.vlaamsparlement.be/parlementaire-d...,N-VA,34.0
3,1424254,2020-09-28,2020-10-29,Bart Somers,Gemeente- en OCMW-raadsleden - Inzagerecht v...,Vraag en antwoord,Lokale Overheden,Brecht Warnez,https://www.vlaamsparlement.be/parlementaire-d...,cd&v,31.0
4,1423090,2020-09-22,2020-10-27,Ben Weyts,Lerarenopleiding - Instaptoets,Vraag en antwoord,Onderwijs en Vorming,Loes Vandromme,https://www.vlaamsparlement.be/parlementaire-d...,cd&v,35.0


In [14]:
# Convert the datetime64[ns] to date() for easier handling in dash application
details_questions_term_df['datum gesteld'] = details_questions_term_df['datum gesteld'].dt.date
details_questions_term_df['datum beantwoord'] = details_questions_term_df['datum beantwoord'].dt.date

In [15]:
# Function to create markdown-style links
def create_markdown_link(row):
    return f"[{row['onderwerp']}]({row['url']})"

# Apply the function to create a new column 'onderwerp_markdown'
details_questions_term_df["onderwerp"]= details_questions_term_df.apply(create_markdown_link, axis=1)

details_questions_term_df = details_questions_term_df.drop(['url'], axis=1)

In [16]:
details_questions_term_df.head()

,id,datum gesteld,datum beantwoord,minister,onderwerp,documenttype,thema,vraagsteller,vraagsteller_partij,termijn antwoord
0,1363595,2020-01-29,2021-02-05,Wouter Beke,[Baarmoederhalskanker - Regionale verschille...,Vraag en antwoord,Gezondheid,Freya Saeys,Open Vld,373.0
1,1424087,2020-09-24,2020-11-16,Hilde Crevits,[Bedrijfssteun - Evaluatie (3)](https://www....,Vraag en antwoord,Economie,Maurits Vande Reyde,Open Vld,53.0
2,1424234,2020-09-28,2020-11-01,Ben Weyts,[Boekencheck 2020 - Resultaten](https://www....,Vraag en antwoord,Onderwijs en Vorming,Annabel Tavernier,N-VA,34.0
3,1424254,2020-09-28,2020-10-29,Bart Somers,[Gemeente- en OCMW-raadsleden - Inzagerecht ...,Vraag en antwoord,Lokale Overheden,Brecht Warnez,cd&v,31.0
4,1423090,2020-09-22,2020-10-27,Ben Weyts,[Lerarenopleiding - Instaptoets](https://www...,Vraag en antwoord,Onderwijs en Vorming,Loes Vandromme,cd&v,35.0


Finally, a government notice ([Omzendbrief VR 2019/11](https://overheid.vlaanderen.be/sites/default/files/media/documenten/regelgeving/VR%202019%201705%20MED.0216-2%20Omzendbrief%20beleid%20en%20regelgevingsprocessen%20%20-%20bijlage.pdf)) holds the rules relevant for parliamentary questions (Section 6.4, p. 87 ff.). With regard to the term to reply questions, the notice states the following (translated):

*The statutory response period of the Flemish Parliament is twenty working days and starts on the first working day after the week in which the question was delivered to the cabinet. This period is suspended during the second week of the Easter recess, from the beginning of the summer recess until August 31, and from December 25 to January 2.*

Therefore, we create a function below to determine the last day of this statutary response date. We chose a manual approach, since even with dedicated package (e.g. `workalander` and `holidays`), it is not so easy to extract the dates of the Eastern holidays (see [here](https://www.schoolvakantieseuropa.be/schoolvakanties-belgie/paasvakantie/) and [here](https://nl.wikipedia.org/wiki/Paasvakantie)). Dedicated webpages of the Flemish government indicate the holidays for the [recent past](https://onderwijs.vlaanderen.be/nl/schoolvakanties/schoolvakanties-vorige-schooljaren) and the [near future](https://www.vlaanderen.be/onderwijs-en-vorming/wat-mag-en-moet-op-school/schoolvakanties-vrije-dagen-en-afwezigheden/schoolvakanties).

In [17]:
# Function to obtain last week of the easter break for a given year
def obtain_last_week_easter_break(year):
    # Create an instance of the Belgium class
    belgium_calendar = Belgium()
    
    # Get date of Easter
    easter_date = belgium_calendar.get_easter_sunday(year)

    # If Easter is in March, the Easter break starts on Easter Monday
    if easter_date.month == 3:
        start_date = easter_date + timedelta(days=1)    

    # If Easter is after April 15, the Easter break starts on the second Monday before Easter    
    elif easter_date > easter_date.replace(month=4, day=15):
            start_date = easter_date - timedelta(weeks=1, days=easter_date.weekday())
        
    else:
        # If Easter is in the first half of April, the Easter break starts on the first Monday of April
        start_date = easter_date.replace(month=4, day=1) - timedelta(days=easter_date.replace(month=4, day=1).weekday())

    # Obtain relevant dates for last week, i.e. week after start
    start_last_week = start_date + timedelta(weeks=1)
    end_date_last_week = start_date + timedelta(weeks=2)  

    last_week_range = pd.date_range(start = start_last_week, end = end_date_last_week - timedelta(days=1))

    last_week_list = [date.date() for date in last_week_range]
    
    return last_week_list

In [18]:
# Function to calculate non-working days for a given year, considering the provided rules
def obtain_excluded_days(year):
    # =============================================================================
    # Initialize non-working-days
    # =============================================================================
    # Create an instance of the Belgium class
    belgium_calendar = Belgium()

    # Get public holidays of current year
    # No need to get days of next year: Jan 1 and 2 are covered by christmass break, 
    # and closest holiday after is Easter, which is too far of anyway
    public_holidays = belgium_calendar.holidays(year)
    # Maintain only date elements, not label to indicate holiday
    public_holidays = [i[0] for i in public_holidays]

    # Get last week of easter holiday of current year 
    # (term for even question at end of year will not reach nex Easter) 
    last_easter_week = obtain_last_week_easter_break(year)

    # define break from 21 July (included) up until 31 August (included)
    summer_reces = [date.date() for date in pd.date_range(f'{year}-07-21', f'{year}-08-31')]
    # define break from 25 December (included) up until 2 January (included)
    christmas_break = [date.date() for date in pd.date_range(f'{year}-12-25', f'{year + 1}-01-02')]
    
    # combine
    excluded_dates = public_holidays + last_easter_week + summer_reces + christmas_break

    return excluded_dates

In [19]:
# Obtain earliest year and latest year in current dataset.
# Use 'dropna' to avoid errors for missing elements
# use 'datum gesteld' to determine earliest_year since this is the first date registered.
# Similarly, use 'datum beantwoord' to determine latest_year since this is the lateste date registered.
earliest_year = details_questions_term_df['datum gesteld'].dropna().min().year
latest_year = details_questions_term_df['datum beantwoord'].dropna().max().year

In [20]:
# Obtain all relevant dates that should be excluded over the entire relevant period
excluded_days_relevant_all = []
for year in range(earliest_year, latest_year + 1):
    excluded_days_relevant = obtain_excluded_days(year)
    excluded_days_relevant_all.extend(excluded_days_relevant)

# # inspect results
# excluded_days_relevant_all

In [24]:
# Function to calculate working days considering the provided rules
def calculate_working_days(row, excluded_days):
    # Initialize data range
    question_delivery_date = row['datum gesteld']
    end_date = row['datum beantwoord']
    
    # Check if question_delivery_date and end_date are valid (not NaT)
    if pd.isna(question_delivery_date) or pd.isna(end_date):
        return np.nan  # Or any other appropriate value for missing data

    # Move to the end of the week of the question delivery
    end_of_week = question_delivery_date + timedelta(days=(4 - question_delivery_date.weekday() + 7) % 7)

    # Find the first working day after the end of the week
    start_date = end_of_week + timedelta(days=1)
    while start_date.weekday() > 4 or start_date in excluded_days:
        start_date += timedelta(days=1)
    
    # Generate an initial range of dates between start_date and end_date
    date_range = pd.date_range(start=start_date, end=end_date)

    # Filter weekdays (Monday=0, Sunday=6)
    working_days = np.isin(date_range.weekday, [0, 1, 2, 3, 4])
    
    # Exclude non-working days and count working days
    working_days = working_days & ~np.isin(date_range, excluded_days)
    working_days_count = np.sum(working_days)

    return working_days_count

In [25]:
# Create a new column 'termijn antwoord (werkdagen)' using the apply function with excluded_days as an argument
details_questions_term_df['termijn antwoord (werkdagen)'] = details_questions_term_df.apply(
    calculate_working_days, axis=1, args=(excluded_days_relevant_all,)
)


In [26]:
details_questions_term_df.head()

,id,datum gesteld,datum beantwoord,minister,onderwerp,documenttype,thema,vraagsteller,vraagsteller_partij,termijn antwoord,termijn antwoord (werkdagen)
0,1363595,2020-01-29,2021-02-05,Wouter Beke,[Baarmoederhalskanker - Regionale verschille...,Vraag en antwoord,Gezondheid,Freya Saeys,Open Vld,373.0,265.0
1,1424087,2020-09-24,2020-11-16,Hilde Crevits,[Bedrijfssteun - Evaluatie (3)](https://www....,Vraag en antwoord,Economie,Maurits Vande Reyde,Open Vld,53.0,36.0
2,1424234,2020-09-28,2020-11-01,Ben Weyts,[Boekencheck 2020 - Resultaten](https://www....,Vraag en antwoord,Onderwijs en Vorming,Annabel Tavernier,N-VA,34.0,20.0
3,1424254,2020-09-28,2020-10-29,Bart Somers,[Gemeente- en OCMW-raadsleden - Inzagerecht ...,Vraag en antwoord,Lokale Overheden,Brecht Warnez,cd&v,31.0,19.0
4,1423090,2020-09-22,2020-10-27,Ben Weyts,[Lerarenopleiding - Instaptoets](https://www...,Vraag en antwoord,Onderwijs en Vorming,Loes Vandromme,cd&v,35.0,22.0


In [27]:
## Save details_questions_term_df for later use
# 1. Save as pkl
with open('../data/details_questions_term_df.pkl', 'wb') as file:
    pickle.dump(details_questions_term_df, file)

# 2. Save as csv
details_questions_term_df.to_csv(path_or_buf = '../data/details_questions_term_df.csv',
                               sep = ";",
                               encoding = "utf-16", # to ensure trema's are well handled (e.g. Koen Daniëls)
                               index = False)


# Dump

In [ ]:
# question_delivery_date = details_questions_term_df.iloc[0]["datum gesteld"]
# question_delivery_date



In [ ]:
# # Move to the end of the week of the question delivery
# end_of_week = question_delivery_date + timedelta(days=(4 - question_delivery_date.weekday() + 7) % 7)
# end_of_week


In [ ]:
# # Find the first working day after the end of the week
# start_date = end_of_week + timedelta(days=1)
# while start_date.weekday() > 4 or start_date in excluded_days_relevant_all:
#     start_date += timedelta(days=1)

In [ ]:
# start_date

In [ ]:
# start_date.weekday()

In [ ]:
# belgium_calendar = Belgium()
# public_holidays = belgium_calendar.holidays(2022)
# public_holidays

# [i[0] for i in public_holidays]

In [ ]:
# test = obtain_last_week_easter_break(2023)
# test
# [date.date() for date in test]

In [ ]:
# # Function to calculate working days considering the provided rules
# def calculate_working_days(row):
#     # =============================================================================
#     # Initialize data range
#     # =============================================================================
#     start_date = row['datum gesteld']
#     end_date = row['datum beantwoord']
    
#     # Check if start_date and end_date are valid (not NaT)
#     if pd.isna(start_date) or pd.isna(end_date):
#         return np.nan  # Or any other appropriate value for missing data

    
#     # Generate an initial range of dates between start_date and end_date
#     date_range = pd.date_range(start=start_date, end=end_date)

#     # Filter weekdays (Monday=0, Sunday=6)
#     working_days = np.isin(date_range.weekday, [0, 1, 2, 3, 4])

#     # =============================================================================
#     # Initialize non-working-days
#     # =============================================================================
#     # Create an instance of the Belgium class
#     belgium_calendar = Belgium()

#     # Get public holidays of current year
#     # No need to get days of next year: Jan 1 and 2 are covered by christmass break, 
#     # and closest holiday after is Easter, which is too far of anyway
#     public_holidays = belgium_calendar.holidays(start_date.year)
#     # Maintain only date elements, not label to indicate holiday
#     public_holidays = [i[0] for i in public_holidays]

#     # Get last week of easter holiday of current year 
#     # (term for even question at end of year will not reach nex Easter) 
#     last_easter_week = obtain_last_week_easter_break(start_date.year)

#     # define break from 21 July (included) up until 31 August (included)
#     summer_reces = [date.date() for date in pd.date_range(f'{start_date.year}-07-21', f'{start_date.year}-08-31')]
#     # define break from 25 December (included) up until 2 January (included)
#     christmas_break = [date.date() for date in pd.date_range(f'{start_date.year}-12-25', f'{start_date.year + 1}-01-02')]
    
#     # combine
#     excluded_dates = [public_holidays + last_easter_week + summer_reces + christmas_break]

#     # Exclude non-working days based on the provided rules
#     working_days = working_days & ~np.isin(date_range, excluded_dates)
#     # =============================================================================
#     # Exclude non-working days and count working days
#     # =============================================================================
#     working_days_count = np.sum(working_days)

#     return working_days_count

In [ ]:
# import pandas as pd
# summer_reces = [date.date() for date in pd.date_range('2023-07-21', '2023-08-31')]
# summer_reces

In [ ]:
# belgium_calendar.holidays(2023) + belgium_calendar.holidays(2023 + 1) 

In [ ]:
# from workalendar.europe import Belgium
# from datetime import timedelta

In [ ]:
# cal = Belgium()
# cal.EASTER_METHOD

In [ ]:
# from workalendar.europe import Belgium

# # Create an instance of the Belgium class
# belgium_calendar = Belgium()

# # Specify the year for which you want to get Easter holidays
# year = 2022

# # Get Easter Monday
# easter_monday = belgium_calendar.get_easter_monday(year)
# easter_monday

In [ ]:
# cal.holidays()

In [ ]:
# # Create an instance of the Belgium class
# belgium_calendar = Belgium()

# # List all attributes of the Belgium instance
# attributes = dir(belgium_calendar)
# # Print the list of attributes
# print(attributes)

In [ ]:
# from datetime import date
# import holidays
# be_holidays = holidays.BE()


In [ ]:
# for k,v in be_holidays.items():
#     print (k)

In [ ]:
# len(be_holidays)
# be_holidays

In [ ]:
# # len(be_holidays)
# print(be_holidays)